In [ ]:
%pip install pooch
%pip install textx

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import Audio
from os.path import getsize
from scipy.datasets import face
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
%matplotlib inline

# Instructions 

This is an Jupyter notebook. Notebooks makes it easy to run small snippets in
  Python code to see what happens! Here are some useful commands to use:
   
  1. Click on a cell (will have a green outline around it) and press
      <kbd>Shift</kbd> + <kbd>Enter</kbd> to run the code in the cell. Make sure you run the top cell
      first, because it contains important import statements!
      
  2. You can also run a cell, or all the cells, using the 'Cell' menu up top.
  
  3. Save your work with <kbd>Ctrl</kbd> + <kbd>S</kbd>, or in the File menu.
  
  If you have any questions, just ask!

# 1 - Images

## 1.1 Images as matrices of numbers

For the following section, create some pixel art in greyscale. That means
using numbers from 0-1, 0 being totally black and 1 being totally white.

Don't spend tooooo much time on this, but make sure your picture has some
different shades of grey. Press <kbd>Shift</kbd> + <kbd>Enter</kbd> while hovering on this cell to
update your changes.

In [ ]:
# Feel free to modify the numbers below to change the colors
my_image = np.array([
    [1, 0.75, 0.5, 0.25, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.2],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.2, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.1, 0.2, 0.3, 0.4],
    [0.0, 0.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    [0.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
])
plt.imshow(my_image, cmap="gray")

## 1.2 Image Masking

What if I wanted to make put a watermark or a YouTube annotation on the image? I have to overlay my image on top of your image. Let's say that my annotation, which is 9 pixels, looks like this:

In [ ]:
annotation = np.array([
    [0.5, 0.9, 0.5],
    [0.9, 0.5, 0.9],
    [0.5, 0.9, 0.5]
])
plt.imshow(annotation, cmap="gray")

Overlaying the images takes a couple steps:

1. Create an image that is totally blank except for my annotation, exactly where I want it. 
2. Create a mask with 1s and 0s that correspond to the part of the image that takes up the annotation. Multiplying this mask with the image will remove that part of the original image (an AND operation).
3. "Add" the two images together (OR operation). The final result will have your image with my annotation in the corner!

### Step 1: Create annotation image

In [ ]:
print("Here is an image with all zeros except for my annotation:")

annotation_in_corner = np.array([
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.9, 0.5],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 0.5, 0.9],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.9, 0.5],
])
plt.imshow(annotation_in_corner, cmap="gray")

### Step 2: Create Bitmask and multiply (AND) by the original image

In [ ]:
print(
"""
This bitmask will keep all of the pixels of your image except for the bottom 9 in the corner when I multiply it with your image.
In effect, the pixel is nonzero if my pixel AND yours are nonzero.
""")

bitmask = np.array([
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
])
plt.imshow(bitmask, cmap="gray")

In [ ]:
print("Now, let's multiply my bitmask and your image together! This is what the array looks like:")

my_image_masked = my_image * bitmask
print(my_image_masked)
plt.imshow(my_image_masked, cmap="gray")

### Step 3: Finally, let's create the end image!

In [ ]:
result = my_image_masked + annotation_in_corner

plt.imshow(result, cmap="gray")

## 1.3 Image Transformations

Now that we have an image, we can make modifications to it by changing each pixel's value in a systematic way. 

We can brighten, darken, change the white balance, or change the color of the image with simple lines of code!

Let's apply some transformations to your image! Try to guess what each code block is doing before you run it.

### 1.3.0 The original image

In [ ]:
my_image = result
print(my_image)
plt.imshow(result, cmap="gray")

### 1.3.1 What would IPython show?!?!

This is the first transformation. Run the code block when you're ready.

In [ ]:
my_image2 = np.round(my_image)
    
print(".ot resolc saw ti eno hcihw no desab ,etihw ro kcalb rehtie ot lexip hcae strevnoc noitamrofsnart\
 siht rof edoc ehT"[::-1])
print(my_image2)
plt.imshow(my_image2, cmap="gray")

### 1.3.2 What would IPython show?

In [ ]:
my_filter = -0.5
my_image3 = np.clip(my_image + my_filter, 0, 1)

print("Here is what the matrix looks like:")
print(my_image3)
print("?neppah lliw kniht uoy od tahW .evitisop ti gnikam neve ro elbairav retlif_ym eht gnignahc\
 yrT !ti ot tnuoma (evitagen() tnatsnoc a sdda dna rebmun hcae sekat tI .retlif gninekrad a si eno sihT"[::-1])

plt.imshow(my_image3, cmap="gray")

#### 1.3.3 Filtering with full-sized images

Now let's work with a real image. This image has way more pixels than your pixel art, but the
structure is exactly the same! (note: not *exactly* the same, because the numbers are now from 0-255).

Let's apply some of the same "filters" that we applied to your pixel art. We'll start with a greyscale version.

In [ ]:
raccoon_gray = face(gray=True)
plt.imshow(raccoon_gray, cmap="gray")

In [ ]:
# Run this code to apply the black and white filter
raccoon_gray_bw = raccoon_gray.copy()
for row_index, row in enumerate(raccoon_gray_bw):
    for pixel_index, pixel in enumerate(row):
        raccoon_gray_bw[row_index][pixel_index] = 255 if pixel > 128 else 0
        
plt.imshow(raccoon_gray_bw, cmap="gray")

In [ ]:
# add a darkening (negative) or lightening (positive) amount here!
# remember that the values range from 0-255 now.
my_filter = -100

# Run this code to apply the darkening and lightening filter.
raccoon_gray_darken = np.clip(raccoon_gray + my_filter, 0, 255)


plt.imshow(raccoon_gray_darken, cmap="gray")

### 1.3.4 Working with color images

For our final step, let's work with color. Now, instead of there being one number for each pixel, there are three. These are the Red, Green, and Blue (RGB) values that you've probably heard of. 

We can manipulate these values in a similar way as with greyscale, but now we have even more creative control. Let's give our raccoon a nice Instagram filter.

In [ ]:
# First, view the original picture in all its glory!
raccoon = face()
plt.imshow(raccoon)
# Notice that each "item" of the data grid is now 3 values
# corresponding to Red, Green, and Blue values between 0 and 255
# The code below prints the top left corner, in a 3x3 region
print(raccoon[:3, :3, ...])

In [ ]:
# Now, it's time to modify the color values. We will boost some colors by increasing their magnitude.
raccoon_r, raccoon_g, raccoon_b = np.dsplit(raccoon.astype(int), raccoon.shape[-1])

# play around with these values if you want!
lighten = -20
red_boost = -10 + lighten
green_boost = 45 + lighten
blue_boost = 30 + lighten

raccoon_filtered = np.clip(np.dstack([
    raccoon_r + red_boost,
    raccoon_g + green_boost,
    raccoon_b + blue_boost
]), 0, 255).astype(np.uint8)
plt.imshow(raccoon_filtered)

## 1.4 Image Compression by Grouping Similar Pixels

The raccoon image above takes $768 \times  1024 \times 3 = 2359296 \text{ bytes} = 2.36 \text{ MB}$ (each number is `uint8`, which is 1 byte).

That's pretty big for one picture! To put that in perspective, a one minute long video running at 24 frames per second using the above encoding will have a size of $2.36 * 24 * 60 = 3398.4 \text{ MB} = 3.4 \text{ GB}$!

One way to compress images is by reducing the amount of unique colors in the image. 

The code below groups an image's pixels by color into n clusters, and it replaces each pixel in the image with the center of the cluster that the pixel belongs to.

In [ ]:
def compress_im(im: np.ndarray, n_clusters: int, samples: int = 1000) -> np.ndarray:
    '''Compresses color image by finding the top n_groups of pixels, 
    then replacing all pixels with their closest neighbor in that group.
    
    Don't worry about the specifics of this function.
    '''
    # house keeping code to massage the data formats
    if im.dtype == np.uint8:
        im = im.astype(np.float64) / 255        
    shape = im.shape    
    im_X = im.reshape(shape[0]*shape[1], 3)
    im_X_to_sample = im_X.copy()
    shuffle(im_X_to_sample)
    im_X_sampled = im_X_to_sample[:samples]
    
    # using KMeans algorithm to find n clusters
    kmeans = KMeans(n_clusters=n_clusters, n_init=10).fit(im_X_sampled)
    
    # reconstructing compressed image with the found clusters
    centers = kmeans.cluster_centers_
    group_assignments = kmeans.predict(im_X)
    im_Y = np.take(centers, group_assignments, axis=0)
    im_compressed = im_Y.reshape(shape[0], shape[1], 3)
    return im_compressed

In [ ]:
print(f"Original image has roughly {raccoon.shape}")

Vary the number of clusters and see how the results change!

1. What happens when you just have 1 cluster? 2?
2. How many clusters do you need to have more than "1" dimension of color?
3. What's the minimum number of clusters so that you can't tell the difference between the original and the compressed image?

In [ ]:
n_clusters = 10
compressed_raccoon = compress_im(raccoon, n_clusters)
plt.imshow(compressed_raccoon)
print(f"Compressed image has roughly {compressed_raccoon.shape}")

# 2 - Audio

Like images, sounds are also just arrays of numbers that describe the "intensity" of the sound at each step. 

Sound files need to specify a "sample rate" in the unit of hertz (1/second), which describes how frequent each "intensity" step is. 

For example a sound file with a sample rate of 10 means its numbers describe sound intensities that change every 0.1s.

Standard quality audio often has a sample rate of 44100 = 44k. Meaning the sound intensity can very 44 thousand times in a second!

In [ ]:
from textx.metamodel import metamodel_from_file

In [ ]:
frequency_table = {
    'A#0': 29.14,
    'A#1': 58.27,
    'A#2': 116.54,
    'A#3': 233.08,
    'A#4': 466.16,
    'A#5': 932.33,
    'A#6': 1864.66,
    'A#7': 3729.31,
    'A#8': 7458.62,
    'A0': 27.5,
    'A1': 55.0,
    'A2': 110.0,
    'A3': 220.0,
    'A4': 440.0,
    'A5': 880.0,
    'A6': 1760.0,
    'A7': 3520.0,
    'A8': 7040.0,
    'Ab0': 25.96,
    'Ab1': 51.91,
    'Ab2': 103.83,
    'Ab3': 207.65,
    'Ab4': 415.3,
    'Ab5': 830.61,
    'Ab6': 1661.22,
    'Ab7': 3322.44,
    'Ab8': 6644.88,
    'B0': 30.87,
    'B1': 61.74,
    'B2': 123.47,
    'B3': 246.94,
    'B4': 493.88,
    'B5': 987.77,
    'B6': 1975.53,
    'B7': 3951.07,
    'B8': 7902.13,
    'Bb0': 29.14,
    'Bb1': 58.27,
    'Bb2': 116.54,
    'Bb3': 233.08,
    'Bb4': 466.16,
    'Bb5': 932.33,
    'Bb6': 1864.66,
    'Bb7': 3729.31,
    'Bb8': 7458.62,
    'C#0': 17.32,
    'C#1': 34.65,
    'C#2': 69.3,
    'C#3': 138.59,
    'C#4': 277.18,
    'C#5': 554.37,
    'C#6': 1108.73,
    'C#7': 2217.46,
    'C#8': 4434.92,
    'C0': 16.35,
    'C1': 32.7,
    'C2': 65.41,
    'C3': 130.81,
    'C4': 261.63,
    'C5': 523.25,
    'C6': 1046.5,
    'C7': 2093.0,
    'C8': 4186.01,
    'D#0': 19.45,
    'D#1': 38.89,
    'D#2': 77.78,
    'D#3': 155.56,
    'D#4': 311.13,
    'D#5': 622.25,
    'D#6': 1244.51,
    'D#7': 2489.02,
    'D#8': 4978.03,
    'D0': 18.35,
    'D1': 36.71,
    'D2': 73.42,
    'D3': 146.83,
    'D4': 293.66,
    'D5': 587.33,
    'D6': 1174.66,
    'D7': 2349.32,
    'D8': 4698.63,
    'Db0': 17.32,
    'Db1': 34.65,
    'Db2': 69.3,
    'Db3': 138.59,
    'Db4': 277.18,
    'Db5': 554.37,
    'Db6': 1108.73,
    'Db7': 2217.46,
    'Db8': 4434.92,
    'E0': 20.6,
    'E1': 41.2,
    'E2': 82.41,
    'E3': 164.81,
    'E4': 329.63,
    'E5': 659.25,
    'E6': 1318.51,
    'E7': 2637.02,
    'E8': 5274.04,
    'Eb0': 19.45,
    'Eb1': 38.89,
    'Eb2': 77.78,
    'Eb3': 155.56,
    'Eb4': 311.13,
    'Eb5': 622.25,
    'Eb6': 1244.51,
    'Eb7': 2489.02,
    'Eb8': 4978.03,
    'F#0': 23.12,
    'F#1': 46.25,
    'F#2': 92.5,
    'F#3': 185.0,
    'F#4': 369.99,
    'F#5': 739.99,
    'F#6': 1479.98,
    'F#7': 2959.96,
    'F#8': 5919.91,
    'F0': 21.83,
    'F1': 43.65,
    'F2': 87.31,
    'F3': 174.61,
    'F4': 349.23,
    'F5': 698.46,
    'F6': 1396.91,
    'F7': 2793.83,
    'F8': 5587.65,
    'G#0': 25.96,
    'G#1': 51.91,
    'G#2': 103.83,
    'G#3': 207.65,
    'G#4': 415.3,
    'G#5': 830.61,
    'G#6': 1661.22,
    'G#7': 3322.44,
    'G#8': 6644.88,
    'G0': 24.5,
    'G1': 49.0,
    'G2': 98.0,
    'G3': 196.0,
    'G4': 392.0,
    'G5': 783.99,
    'G6': 1567.98,
    'G7': 3135.96,
    'G8': 6271.93,
    'Gb0': 23.12,
    'Gb1': 46.25,
    'Gb2': 92.5,
    'Gb3': 185.0,
    'Gb4': 369.99,
    'Gb5': 739.99,
    'Gb6': 1479.98,
    'Gb7': 2959.96,
    'Gb8': 5919.91,
    'N': 0 # nothing
}

music_metamodel = metamodel_from_file("music.tx")

## 2.1 Sounds as numbers in time domain

Instructions:
1. Use the following code to visualize and play the "A" chord. 
2. Vary the sample rate to higher and lower numbers - can you hear the difference?
3. Change the frequencies array to a sequence of other keys (see a mapping of keys to frequencies [here](https://pages.mtu.edu/~suits/notefreqs.html))

In [ ]:
# Play around with these values!
frequencies = [440, 554.37, 659.25]
sample_rate = 44100

# Constructing audio
samples = np.array([np.sin(np.linspace(0, 2 * np.pi * frequency, sample_rate)) for frequency in frequencies])

# Displaying plot
data_rate = sample_rate
data = np.sum(samples, axis=0)
plt.plot(np.arange(data.shape[0]), data, alpha=0.3)
plt.title("Time Domain Intensities")
plt.ylabel("Intensity")
plt.xlabel("Time step")

Audio(np.concatenate(list(samples) + [np.sum(samples, axis=0)]), rate=sample_rate)

To visualize the frequencies of the audio above, we will convert 0.1s "chunks" of the song into the frequency domain.

In [ ]:
def to_chunks(data: np.ndarray, chunks_per_second: int = 1, datarate: int = 44100) -> np.ndarray:
    num_chunks = int(round(chunks_per_second * len(data) / datarate))
    chunks = [data[int(datarate / chunks_per_second * i) : int(datarate / chunks_per_second * (i + 1))] for i in range(num_chunks)]
    frequencies = np.asarray([np.abs(np.fft.rfft(chunk)) for chunk in chunks]).T
    return frequencies

In [ ]:
plt.imshow(20*np.log10(to_chunks(data, 1, sample_rate))[:1000], interpolation="none", aspect="auto", cmap='nipy_spectral')

## 2.2 Sounds as numbers in frequency domain

"Frequency" encoding is very similar to the dictionary-based encoding scheme above. Instead of each key referring to a word, each key here refers to a "chunk" of sound played at the specified frequency of that key.

1. Look at `lullaby.txt` for the frequency (keys) encoding of the lullaby.
2. Run the following code to listen to the audio file.

In [ ]:
def load_song(filename):
    return music_metamodel.model_from_file(filename)

def song_to_frequencies(song):
    parts = {}
    for part in song.parts:
        parts[part.name] = [
            (frequency_table[note.note], note.duration) for note in part.notes
        ]
    return parts

def frequencies_to_stream(freq, samplerate, bpm):
    # 1 beat / beats per minute * min per second * samples per second
    data = []
    for f, beats in freq:
        seconds = beats / bpm * 60
        data.append(np.cos(np.linspace(0, 2 * np.pi * f * seconds, int(round(samplerate * seconds)))))
    return np.concatenate(data)

def smooth(data, length):
    return np.convolve(np.ones(length) / length, data)

In [ ]:
length = 256
sampling_rate = 44100*16

# Change this to change the number of beats per minute
beats_per_minute = 100

song = load_song("lullaby.txt") # change the file name to whatever you created
frequencies = song_to_frequencies(song)

# Replace the names with the part names you chose
song_parts = ["melody", "harmony", "left_hand_high", "left_hand_low"]
song_data = sum(frequencies_to_stream(frequencies[name], sampling_rate, beats_per_minute) for name in song_parts)
Audio(smooth(song_data, 256)[::sampling_rate // 44100], rate=44100)

Run the following code to visualize "chunks" of the lullaby in the frequency domain

What does it look like?

In [ ]:
sheet_music = np.log10(to_chunks(song_data, 1, data_rate) + 0.001)[:120][::-1]
plt.imshow(sheet_music, interpolation="none", aspect="auto", cmap='nipy_spectral')

## Compression
Below is the compression ratio for the song. Notice that if we store the file as text (as a list of frequencies) rather than raw audio data (which is a list of bytes over time), we can reduce the size of the data.

In [ ]:
lullaby_size = getsize("lullaby.txt")

print(f"Sheet music size: {song_data.nbytes} bytes")
print(f"Audio data size: {lullaby_size} bytes")
print(f"Total compression ratio: {song_data.nbytes / lullaby_size}")

Try modifying the file `lullaby.txt` with your own notes or writing your own file!

Notes are written in standard notation (i.e. Bb3) means the third B-flat. Rests are written as N.

Ab3 4 means play the third A-flat for 4 beats.

Each part must have the same number of beats.

## Edit lullaby.txt like:
```
[part_name_1]
notes

[part_name_2]
notes
```

Run the previous two sections of the notebook again, replacing `melody`, `harmony` and the other names with your part names.

## 2.3 Compression in JPEG (Optional)

Similar to audio, there can also be frequencies found in 2D images. It turns out, we can describe any 8x8 image as the coefficients of 64 different "basis images", each with different frequencies horizontally and vertically. 

Below is a look at how analyzing these 2d frequencies are used in practice, specifically for JPEG compression!

In [ ]:
# Here is what some of those 2d frequencies look like.
# play around with the x and y frequencies by 
# toggling x_factor and y_factor!

x_factor = 7 # should be between 0 and 8.
y_factor = 7 # should be between 0 and 8.
base_freq = np.pi / 16.0

# creates the (i, j) basis image of the discrete cosine transform (DCT) used in JPEG
def basis_image(x_fact: int, y_fact: int) -> np.ndarray:
    image = np.array([
        [np.cos(base_freq * x_fact * (2*j + 1)) * np.cos(base_freq * y_fact * (2*i + 1)) for j in range(8)] 
        for i in range(8)
    ])

    image = 0.5 * (image + 1) # scaling / shifting so every value is between 0 and 1
    return image

plt.imshow(basis_image(x_factor, y_factor), cmap="gray")


When we linearly combine two or more of these basis images, we can get really complex patterns!

This is useful in compression, as we can combine these images to make a lot of the same patterns you would see in an actual image.

In [ ]:
basis_1 = basis_image(4, 1)
basis_2 = basis_image(5, 5)

alpha = 0.3 # what fraction of the first image to add

plt.imshow(alpha * basis_1 + (1 - alpha) * basis_2, cmap="gray")

JPEG works by breaking up every 8x8 pixel chunk of an image, and calculating the 64 coefficients of these basis images that would all add together to recreate it.

You may be wondering where the compression in this is-- the original chunk had 64 pixels, so wouldn't storing the 64 coefficients not be any better?

This is exactly right. Where JPEG gets its compression is by throwing away coefficients it doesn't need. For example, if an image doesn't recquire high-detailed texture, one can throw away the high frequency images and still have a fairly recognizable overall image. Also, if some of the coefficients are very small, it may be better not to add them to the final JPEG at all.

In [ ]:
import scipy.fftpack as fft

In [ ]:
# CREDIT: Frank Ong for EE 123:
# https://inst.eecs.berkeley.edu/~ee123/sp16/Sections/JPEG_DCT_Demo.html

# Without going into too much detail, this code determines the 64 
# coefficients for each 8x8 chunk, then removes the ones that are not larger
# than some threshold. Once the low coefficients are gone, it recreates the 
# image by adding back all those scaled basis images we calculated earlier.

def dct2(a: np.ndarray) -> np.ndarray:
    return fft.dct(fft.dct(a, axis=0, norm='ortho'), axis=1, norm='ortho')

def idct2(a: np.ndarray) -> np.ndarray:
    return fft.idct(fft.idct(a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

def jpeg_compress_img(im: np.ndarray, threshold: float) -> np.ndarray:
    im_w, im_l = im.shape # dimensions of the image
    im_dct = np.zeros(im.shape) # creating an empty image to store the frequency coefficients
    
    # Do 8x8 DCT on image (in-place), using fancy numpy indexing magic!
    for i in np.arange(0, im_w, 8):
        for j in np.arange(0, im_l, 8):
            im_dct[i:i+8, j:j+8] = dct2(im[i:i+8, j:j+8])
            
    # Applying threshold to each block:
    
    dct_thresh = im_dct * (np.abs(im_dct) > (threshold * np.amax(im_dct)))
    
    percent_nonzeros = float(np.sum(dct_thresh != 0.0) / (im_w * im_l))

    print(f"Keeping only {percent_nonzeros:.2%} of the DCT coefficients")
    
    # getting back the compressed image
    
    im_idct = np.zeros(im.shape)
    for i in np.arange(0, im_w, 8):
        for j in np.arange(0, im_l, 8):
            im_idct[i:i+8, j:j+8] = idct2(dct_thresh[i:i+8, j:j+8] )
            
    # normalize:
    im_idct = im_idct / np.amax(im_idct)
    
    return im_idct

In [ ]:
original_raccoon = raccoon_gray.copy()
plt.imshow(original_raccoon, cmap="gray")

In [ ]:
thresh = 0.3 # increase to decrease quality.
compressed_raccoon = jpeg_compress_img(original_raccoon, thresh)
plt.imshow(compressed_raccoon, cmap="gray")